In [1]:
# importa os pacotes necessários
import numpy as np
import os, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import np_utils
from sklearn import tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog

In [2]:
#funções de leitura e preparação das imagens
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        if i%250 == 0: print('Processed {} of {}'.format(i, count))    
    return data

def show_cats_and_dogs(idx):
    cat = read_image(train_cats[idx])
    dog = read_image(train_dogs[idx])
    pair = np.concatenate((cat, dog), axis=1)
    plt.figure(figsize=(10,5))
    plt.imshow(pair)
    plt.show()
    
# dois exemplos de descritores. Você deve criar outros mais robustos.
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):     
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    #image = cv2.imread(image_file)        
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
        [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [3]:
TRAIN_DIR = 'kaggle/cifar-10/train'
TRAIN_LABELS_DIR = 'kaggle/cifar-10/trainLabels.csv'

ROWS = 128
COLS = 128
CHANNELS = 3
NIM = 1000

image_paths = os.listdir(TRAIN_DIR)

train_images = [read_image(TRAIN_DIR + "/" + i) for i in image_paths][:NIM]
train_labels = pd.read_csv(TRAIN_LABELS_DIR)
del train_labels['id']

#num_classes = 10
#y_train = np_utils.to_categorical(y_train,num_classes)
#y_test = np_utils.to_categorical(y_test,num_classes)

# considera apenas NIM imagens. Para o dataset completo, desconsiderar.
# random.shuffle(train_images)

In [4]:
rawImages = []
descHist = []

count = len(train_images)

for i, image in enumerate(train_images):
    pixels = image_to_feature_vector(image)
    histogram = extract_color_histogram(image)
    
    rawImages.append(pixels)
    descHist.append(histogram)
        
    if i%1000 == 0: print('Processed {} of {}'.format(i, count))
        

Processed 0 of 1000


In [5]:
train_labels = train_labels[:NIM]
train_labels

,label
0,frog
1,truck
2,truck
3,deer
4,automobile
...,...
995,cat
996,dog
997,automobile
998,cat


In [6]:
from sklearn import datasets, svm, metrics

#Avalia o primeiro descritor: as imagens raw
(X_train, X_test, y_train, y_test) = train_test_split(rawImages, train_labels, test_size=0.30, random_state=42)

classifiers = [svm.SVC(gamma=0.001),
               MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1),
               KNeighborsClassifier(5),
               DecisionTreeClassifier(),
               GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    print("did")
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))

C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC
****Results****
did
accuracy: 10.33%


C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier
****Results****
did
accuracy: 10.33%
KNeighborsClassifier
****Results****


C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  del sys.path[0]


did
accuracy: 10.33%
DecisionTreeClassifier
****Results****
did
accuracy: 13.00%
GaussianNB
****Results****


C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


did
accuracy: 8.67%


In [7]:
#Avalia o segundo descritor: color histogram
labels = list(train_labels.label.unique())

(X_train, X_test, y_train, y_test) = train_test_split(descHist, train_labels, test_size=0.25, random_state=42)
classifiers = [LinearSVC(),
               MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1),
               SVC(gamma='auto'),
               KNeighborsClassifier(5),
               DecisionTreeClassifier(),
               GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))   

LinearSVC
****Results****
accuracy: 6.40%


C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier
****Results****
accuracy: 8.40%


C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC
****Results****
accuracy: 8.40%
KNeighborsClassifier
****Results****


C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  del sys.path[0]


accuracy: 8.00%
DecisionTreeClassifier
****Results****
accuracy: 8.40%
GaussianNB
****Results****
accuracy: 12.40%


C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
#Avalia a combinação dos dois primeiros descritores!

#ATENÇÃO: ESTE É APENAS UM CÓDIGO EXEMPLO. VOCÊ DEVE DESENVOLVER
#DESCRITORES MAIS ROBUSTOS, BEM COMO EXPLORAR MELHOR AS MÉTRICAS
#DE AVALIAÇÃO (MATRIZ DE CONFUSÃO, ETC)

trainAux = np.hstack((descHist, rawImages))
(X_train, X_test, y_train, y_test) = train_test_split(trainAux, labels, test_size=0.25, random_state=42)

classifiers = [
    KNeighborsClassifier(17),    
    DecisionTreeClassifier(),
    GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))   

ValueError: Found input variables with inconsistent numbers of samples: [1000, 10]